<a href="https://colab.research.google.com/github/govindmurthi21/statlearning/blob/main/ragsimple2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pymongo[srv] certifi

In [30]:
from google.colab import userdata
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
import certifi
ca = certifi.where()

client = MongoClient(userdata.get("mongo_conn"), server_api=ServerApi('1'), tlsCAFile=ca)
db = client.sample_mflix
collection = db.movies

In [32]:
import requests

hf_token = userdata.get("hugging_token")
embedding_url = "https://api-inference.huggingface.co/pipeline/feature-extraction/sentence-transformers/all-MiniLM-L6-v2"

def generate_embedding(text: str) -> list[float]:
  resp = requests.post(
      embedding_url,
      headers={"Authorization": f"Bearer {hf_token}"},
      json={"inputs": text}
  )

  if resp.status_code != 200:
    raise ValueError(f"Failed with code {resp.status_code}: {resp.text}")

  return resp.json()

print(generate_embedding("This is awesome"))


[-0.061711497604846954, -0.013916903175413609, 0.003470437368378043, -0.056922025978565216, 0.03402159735560417, -0.028545575216412544, 0.08898869156837463, 0.037550318986177444, 0.05078928545117378, -0.006584883667528629, 0.0363127700984478, 0.10030189156532288, 0.014761898666620255, 0.08833707869052887, -0.13524670898914337, 0.00902241189032793, -0.0060308221727609634, 0.03141961246728897, -0.014836126007139683, 0.0121815986931324, -0.07665978372097015, -0.07392250001430511, 0.035814352333545685, -0.002054242417216301, 0.06024349853396416, 0.03384813293814659, -0.008519272319972515, 0.055332597345113754, 0.042162876576185226, -0.04711415246129036, -0.08150138705968857, -0.010802073404192924, 0.06578696519136429, -0.03293659910559654, -0.12828326225280762, 0.0007072318694554269, 0.07937563210725784, 0.0707547664642334, -0.0010973754106089473, 0.04192095249891281, -0.008350654505193233, -0.04695681855082512, 0.02301834337413311, -0.005256174597889185, -0.049484752118587494, 0.087682396

In [33]:
for doc in collection.find({'plot': {"$exists": True}}).limit(50):
  doc['plot_embedding_hf'] = generate_embedding(doc['plot'])
  collection.replace_one({'_id': doc['_id']}, doc)

In [36]:
query = "imaginary characters from outer space at war"

results = collection.aggregate([
    {
        "$vectorSearch": {
            "queryVector": generate_embedding(query),
            "path": "plot_embedding_hf",
            "numCandidates": 100,
            "limit": 4,
            "index": "PlotSemanticSearch"
        }
    }
])

for document in results:
  print(f"Moive Name: {document['title']}. \n Movie Plot: {document['plot']} \n")

Moive Name: Four Sons. 
 Movie Plot: A family saga in which three of a Bavarian widow's sons go to war for Germany and the fourth goes to America, Germany's eventual opponent. 

Moive Name: The Strong Man. 
 Movie Plot: A meek Belgian soldier (Harry Langdon) fighting in World War I receives penpal letters and a photo from "Mary Brown", an American girl he has never met. He becomes infatuated with her by ... 

Moive Name: The Four Horsemen of the Apocalypse. 
 Movie Plot: An extended family split up in France and Germany find themselves on opposing sides of the battlefield during World War I. 

Moive Name: In the Land of the Head Hunters. 
 Movie Plot: Original advertising for the film describes it as a drama of primitive life on the shores of the North Pacific... 

